In [10]:
import sqlite3
conn = sqlite3.connect('example.db', isolation_level=None)
conn.execute('CREATE TABLE IF NOT EXISTS cats (name TEXT NOT NULL, birthdate TEXT, fur TEXT, weight_kg REAL) STRICT')

In [11]:
conn.execute('SELECT name FROM sqlite_schema WHERE type="table"').fetchall()

[('cats',)]

In [12]:
conn.execute('PRAGMA TABLE_INFO(cats)').fetchall()

[(0, 'name', 'TEXT', 1, None, 0),
 (1, 'birthdate', 'TEXT', 0, None, 0),
 (2, 'fur', 'TEXT', 0, None, 0),
 (3, 'weight_kg', 'REAL', 0, None, 0)]

Column position The 1 indicates that the column is second in the table. Column numbers are zero based, like Python list indexes, so the first column is at position 0.

Name 'birthdate' is the name of the column. Remember that SQLite column and table names are case insensitive.

Data type 'TEXT' is the SQLite data type of the birthdate column.

Whether the column is NOT NULL The 0 means False and that the column is not NOT NULL (that is, you can put None values in this column).

Default value None is the default value inserted if no other value is specified.

Whether the column is the primary key The 0 means False, meaning this column is not a primary-key column.

In [13]:
conn.execute('INSERT INTO cats VALUES ("Zophie", "2021-01-24", "black", 5.6)')

## Transactions

An INSERT statement begins a transaction, which is a unit of work in a database. Transactions must pass the ACID test, a database concept meaning that transactions are:

Atomic The transaction is carried out either completely or not at all.

Consistent The transaction doesn’t violate constraints, such as NOT NULL rules for columns.

Isolated One transaction doesn’t affect other transactions.

Durable If committed, the transaction results are written to persistent storage, such as the hard drive.

## Preventing SQL injection attacks

Use ? in query string, then pass the variables in a list argument following the query string. The execute() method replaces the ? placeholders in the query string with the variable values after making sure they won’t cause a SQL injection attack

In [14]:
cat_name = 'Zophie'
cat_bday = '2021-01-24'
fur_color = 'black'
cat_weight = 5.6
conn.execute('INSERT INTO cats VALUES (?, ?, ?, ?)', [cat_name, cat_bday, fur_color, cat_weight])

## Reading from database

In [15]:
conn.execute('SELECT * FROM cats').fetchall()

[('Zophie', '2021-01-24', 'black', 5.6),
 ('Zophie', '2021-01-24', 'black', 5.6)]

In [16]:
conn.execute('SELECT rowid, name FROM cats').fetchall()

[(1, 'Zophie'), (2, 'Zophie')]

In [17]:
for row in conn.execute('SELECT * FROM cats'):
     print('Row data:', row)
     print(row[0], 'is one of my favorite cats.')

Row data: ('Zophie', '2021-01-24', 'black', 5.6)
Zophie is one of my favorite cats.
Row data: ('Zophie', '2021-01-24', 'black', 5.6)
Zophie is one of my favorite cats.


In [18]:
conn.execute('SELECT * FROM cats WHERE fur = "black"').fetchall()

[('Zophie', '2021-01-24', 'black', 5.6),
 ('Zophie', '2021-01-24', 'black', 5.6)]

In [19]:
import pprint
matching_cats = conn.execute('SELECT * FROM cats WHERE fur = "black" OR birthdate >= "2024-01-01"').fetchall()
pprint.pprint(matching_cats)

[('Zophie', '2021-01-24', 'black', 5.6), ('Zophie', '2021-01-24', 'black', 5.6)]


In [20]:
conn.execute('SELECT rowid, name FROM cats WHERE name LIKE "Zo%"').fetchall()

[(1, 'Zophie'), (2, 'Zophie')]

## Indexes can potentially increase performance

A SQL index is a data structure that organizes a column’s data. As a result, queries with WHERE clauses that use these columns will perform better.  Index will likely only speed things up if larger databases.

In [21]:
conn.execute('CREATE INDEX idx_name ON cats (name)')

In [22]:
conn.execute('CREATE INDEX idx_birthdate ON cats (birthdate)')

In [23]:
conn.execute('SELECT name FROM sqlite_schema WHERE type = "index" AND tbl_name = "cats"').fetchall()

[('idx_name',), ('idx_birthdate',)]

In [24]:
## Updating data

In [25]:
conn.execute('UPDATE cats SET fur = "gray tabby" WHERE rowid = 1')

In [26]:
conn.execute('SELECT * FROM cats WHERE rowid = 1').fetchall()

[('Zophie', '2021-01-24', 'gray tabby', 5.6)]

## Deleting

In [27]:
conn.execute('DELETE FROM cats WHERE rowid = 1')

In [28]:
conn.execute('SELECT * FROM cats WHERE rowid = 1').fetchall()

[]

In [29]:
conn.rollback()

In [30]:
conn.execute('INSERT INTO cats VALUES ("Zophie", "2021-01-24", "black", 5.6)')

In [31]:
conn.execute('INSERT INTO cats VALUES ("Theon", "2021-01-24", "black", 5.6)')

## Backups

In [32]:
backup_conn = sqlite3.connect('backup.db', isolation_level=None)
conn.backup(backup_conn)

## Altering and dropping tables

In [33]:
conn.execute('SELECT name FROM sqlite_schema WHERE type="table"').fetchall()

[('cats',)]

In [34]:
conn.execute('ALTER TABLE cats RENAME TO felines')

In [35]:
conn.execute('SELECT name FROM sqlite_schema WHERE type="table"').fetchall()

[('felines',)]

In [36]:
conn.execute('PRAGMA TABLE_INFO(felines)').fetchall()[2]

(2, 'fur', 'TEXT', 0, None, 0)

In [37]:
conn.execute('ALTER TABLE felines RENAME COLUMN fur TO description')

In [38]:
conn.execute('PRAGMA TABLE_INFO(felines)').fetchall()[2]

(2, 'description', 'TEXT', 0, None, 0)

In [39]:
conn.execute('SELECT name FROM sqlite_schema WHERE type="table"').fetchall()

[('felines',)]

In [40]:
conn.execute('ALTER TABLE felines RENAME TO cats')

In [41]:
conn = sqlite3.connect('sweigartcats.db', isolation_level=None)
conn.execute('PRAGMA foreign_keys = ON')
conn.execute('CREATE TABLE IF NOT EXISTS vaccinations (vaccine TEXT, date_administered TEXT, administered_by TEXT, cat_id INTEGER, FOREIGN KEY(cat_id) REFERENCES cats(rowid)) STRICT')

In [42]:
conn.execute('INSERT INTO vaccinations VALUES ("rabies", "2023-06-06", "Dr. Echo", 2)')

OperationalError: no such table: main.cats

In [ ]:
conn.execute('SELECT * FROM cats').fetchall()

[('Zophie', '2021-01-24', 'black', 5.6), ('Theon', '2021-01-24', 'black', 5.6)]

In [ ]:
conn.execute('SELECT rowid, name FROM cats').fetchall()

[(2, 'Theon'), (1, 'Zophie')]